In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import seaborn as sns
import plotly as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

Подготовим данные для второго этапа - прогнозирования района, исходя из заданных параметров.

In [3]:
data = pd.read_csv("C:\\Users\\magvl\\блокноты\\final project\\notebook\\data\\train.csv")

In [4]:
data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [5]:
data = data.assign(log_price_doc=np.log1p(data['price_doc']))
data = data.drop('price_doc', axis=1)

In [6]:
data_year = data[['build_count_after_1995', 'build_count_1971-1995','build_count_1946-1970', 
                 'build_count_1921-1945', 'build_count_before_1920', 'build_year']]

In [7]:
set_year = {'build_count_after_1995': 2000, 'build_count_1971-1995': 1983, 'build_count_1946-1970': 1958,
           'build_count_1921-1945': 1933, 'build_count_before_1920': 1915}

In [8]:
data_year_na = data_year[data_year['build_year'].isnull()]
data_year_na['build_year'] = data_year.idxmax(axis=1)
data_year_na['build_year'] = data_year_na['build_year'].map(set_year)
data_year['build_year'] = data_year['build_year'].fillna(data_year_na['build_year'])

In [9]:
data_year.loc[(data_year['build_year'] > 2020)]

,build_count_after_1995,build_count_1971-1995,build_count_1946-1970,build_count_1921-1945,build_count_before_1920,build_year
10089,48.0,52.0,216.0,0.0,0.0,20052009.0
15220,13.0,50.0,264.0,18.0,2.0,4965.0


In [10]:
data_year.loc[(data_year.build_year == 4965), 'build_year'] = 1958
data_year.loc[(data_year.build_year == 20052009), 'build_year'] = 1958

In [11]:
data_year = data_year.fillna(data_year.mean())

In [12]:
data_year.loc[(data_year['build_year'] < 1000)]

,build_count_after_1995,build_count_1971-1995,build_count_1946-1970,build_count_1921-1945,build_count_before_1920,build_year
9441,61.150824,80.130377,141.426413,26.721586,18.899294,0.0
9620,57.000000,58.000000,95.000000,2.000000,1.000000,1.0
9700,61.150824,80.130377,141.426413,26.721586,18.899294,1.0
9745,23.000000,10.000000,7.000000,1.000000,0.000000,1.0
9764,252.000000,130.000000,40.000000,24.000000,13.000000,1.0
...,...,...,...,...,...,...
30394,61.150824,80.130377,141.426413,26.721586,18.899294,1.0
30425,187.000000,131.000000,278.000000,181.000000,23.000000,0.0
30429,61.150824,80.130377,141.426413,26.721586,18.899294,0.0
30430,61.150824,80.130377,141.426413,26.721586,18.899294,0.0


In [13]:
data_year_min = data_year.loc[(data_year['build_year'] < 1000)]
data_year_min['build_year'] = data_year_min.idxmax(axis=1)
data_year_min['build_year'] = data_year_min['build_year'].map(set_year)
data_year['build_year'].loc[(data_year['build_year'] < 1000)] = data_year_min['build_year']

In [14]:
data_year.isnull().sum()

build_count_after_1995     0
build_count_1971-1995      0
build_count_1946-1970      0
build_count_1921-1945      0
build_count_before_1920    0
build_year                 2
dtype: int64

In [15]:
data_year = data_year.fillna(data_year.mean())

In [16]:
data[['build_count_after_1995', 'build_count_1971-1995','build_count_1946-1970', 
     'build_count_1921-1945', 'build_count_before_1920', 'build_year']] = data_year

In [17]:
data[['build_count_after_1995', 'build_count_1971-1995','build_count_1946-1970', 
     'build_count_1921-1945', 'build_count_before_1920', 'build_year']].isnull().sum()

build_count_after_1995     0
build_count_1971-1995      0
build_count_1946-1970      0
build_count_1921-1945      0
build_count_before_1920    0
build_year                 0
dtype: int64

In [18]:
data[['full_sq', 'life_sq','floor', 'max_floor', 'kitch_sq', 'state']].isnull().sum()

full_sq          0
life_sq       6383
floor          167
max_floor     9572
kitch_sq      9572
state        13559
dtype: int64

In [19]:
data_sq = data[['full_sq','life_sq']][data['life_sq'].notna()]

In [20]:
coef = data_sq['full_sq'].mean() / data_sq['life_sq'].mean()

In [21]:
data['life_sq'] = data['life_sq'].fillna(data['full_sq'].mean() / coef)

In [22]:
data = data[['full_sq', 'life_sq', 'floor', 'max_floor', 'build_year',  'num_room', 
             'kitch_sq', 'state', 'sub_area', 'log_price_doc',  'product_type']]

In [34]:
data.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,log_price_doc,product_type
0,43,27.0,4.0,NaN,1983.0,NaN,NaN,NaN,Bibirevo,15.581952,Investment
1,34,19.0,3.0,NaN,1958.0,NaN,NaN,NaN,Nagatinskij Zaton,15.607270,Investment
2,43,29.0,2.0,NaN,1958.0,NaN,NaN,NaN,Tekstil'shhiki,15.555977,Investment
3,89,50.0,9.0,NaN,2000.0,NaN,NaN,NaN,Mitino,16.388123,Investment
4,77,77.0,4.0,NaN,1915.0,NaN,NaN,NaN,Basmannoe,16.608603,Investment


In [23]:
one_hot = pd.get_dummies(data['product_type'], prefix='product_type', drop_first=True, dtype='float32')
data = pd.concat((data.drop('product_type', axis=1), one_hot), axis=1)

In [24]:
data.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,log_price_doc,product_type_OwnerOccupier
0,43,27.0,4.0,NaN,1983.0,NaN,NaN,NaN,Bibirevo,15.581952,0.0
1,34,19.0,3.0,NaN,1958.0,NaN,NaN,NaN,Nagatinskij Zaton,15.607270,0.0
2,43,29.0,2.0,NaN,1958.0,NaN,NaN,NaN,Tekstil'shhiki,15.555977,0.0
3,89,50.0,9.0,NaN,2000.0,NaN,NaN,NaN,Mitino,16.388123,0.0
4,77,77.0,4.0,NaN,1915.0,NaN,NaN,NaN,Basmannoe,16.608603,0.0


In [25]:
data.isnull().sum()

full_sq                           0
life_sq                           0
floor                           167
max_floor                      9572
build_year                        0
num_room                       9572
kitch_sq                       9572
state                         13559
sub_area                          0
log_price_doc                     0
product_type_OwnerOccupier        0
dtype: int64

In [26]:
data['floor'] = data['floor'].fillna(data['floor'].mean())
data['max_floor'] = data['max_floor'].fillna(data['max_floor'].mean())
data['num_room'] = data['num_room'].fillna(data['num_room'].mean())
data['kitch_sq'] = data['kitch_sq'].fillna(data['kitch_sq'].mean())
data['state'] = data['state'].fillna(data['state'].mean())

In [27]:
data.isnull().sum()

full_sq                       0
life_sq                       0
floor                         0
max_floor                     0
build_year                    0
num_room                      0
kitch_sq                      0
state                         0
sub_area                      0
log_price_doc                 0
product_type_OwnerOccupier    0
dtype: int64

In [28]:
data = data[data.full_sq < 500]

In [29]:
data['life_sq'][data['life_sq'] > data['full_sq']] = data['full_sq'][data['life_sq'] > data['full_sq']] * \
                                    (data['life_sq'].mean() / data['full_sq'].mean())

In [30]:
data['kitch_sq'][data['kitch_sq'] > 200] = data['full_sq'][data['kitch_sq'] > 200] * \
                                            data['kitch_sq'].mean() / data['full_sq'].mean()

In [31]:
data = data[data.full_sq > 5]

In [32]:
data

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,log_price_doc,product_type_OwnerOccupier
0,43,27.00000,4.0,12.558974,1983.000000,1.909804,6.399301,2.107025,Bibirevo,15.581952,0.0
1,34,19.00000,3.0,12.558974,1958.000000,1.909804,6.399301,2.107025,Nagatinskij Zaton,15.607270,0.0
2,43,29.00000,2.0,12.558974,1958.000000,1.909804,6.399301,2.107025,Tekstil'shhiki,15.555977,0.0
3,89,50.00000,9.0,12.558974,2000.000000,1.909804,6.399301,2.107025,Mitino,16.388123,0.0
4,77,77.00000,4.0,12.558974,1915.000000,1.909804,6.399301,2.107025,Basmannoe,16.608603,0.0
...,...,...,...,...,...,...,...,...,...,...,...
30466,44,27.00000,7.0,9.000000,1975.000000,2.000000,6.000000,3.000000,Otradnoe,15.816991,0.0
30467,86,59.00000,3.0,9.000000,1935.000000,4.000000,10.000000,3.000000,Tverskoe,17.034386,0.0
30468,45,35.16718,10.0,20.000000,1912.195364,1.000000,1.000000,1.000000,Poselenie Vnukovskoe,15.757264,1.0
30469,64,32.00000,5.0,15.000000,2003.000000,2.000000,11.000000,2.000000,Obruchevskoe,16.418200,0.0


In [33]:
data.to_csv('C:\\Users\\magvl\\блокноты\\final project\\notebook\\data\\data_sub_area.csv', index=False, index_label=False)